# A step-by-step tutorial of pydocker usage

*GPU binding is not presented in this test.*

In [35]:
from dsd.sys.docker import PyDocker, HC, HCP

In [36]:
import socket
try:
    socket.gethostbyname('dockerhost')
    base_url = 'http://dockerhost:4243'
except socket.error:
    base_url = 'unix:///var/run/docker.sock'
    
docker = PyDocker(base_url)

init pydocker


In [37]:
PyDocker

<class dsd.sys.docker.pydocker.PyDocker at 0x7f477843a530>

# 1. Build

default dockerfile name is `Dockerfile`

* Dockerfile:
```
# For Test
FROM dsdgroup/jupyter:cpu
MAINTAINER zy <zy3381@gmail.com>
EXPOSE 9999
```

* returns: build logs

In [47]:
imageTag = 'buildtest'
response = docker.build(tag=imageTag, dockerfilePath='../docker/test/cpu')
if response:
    for line in response:
        print(line)
else:
    print('build failed')

# For Test
FROM dsdgroup/jupyter:cpu
MAINTAINER zy <zy3381@gmail.com>
EXPOSE 9999

{"stream":"Step 1 : FROM dsdgroup/jupyter:cpu\n"}

{"stream":" ---\u003e e21cb4e5f96c\n"}

{"stream":"Step 2 : MAINTAINER zy \u003czy3381@gmail.com\u003e\n"}

{"stream":" ---\u003e Using cache\n"}

{"stream":" ---\u003e 8f02ad69ef0a\n"}

{"stream":"Step 3 : EXPOSE 9999\n"}

{"stream":" ---\u003e Running in 578999735deb\n"}

{"stream":" ---\u003e cbc92338b3f4\n"}

{"stream":"Successfully built cbc92338b3f4\n"}



# 2. list images

In [48]:
pses = docker.images()
for line in pses:
    print line['RepoTags']

buildtest:latest
dsdgroup/jupyter:cpu
guoq/igym:latest
dsdgroup/dsd-console:latest
dsd-console:latest
nvidia/cuda:7.5-runtime
wangjianyong/mxnet-cuda:7.5
wangjianyong/mxnet:latest
wangjianyong/jupyter-cuda:7.5
wangjianyong/jupyter:latest
nvidia/cuda:7.5-cudnn4-devel
nvidia/cuda:7.5-devel
ubuntu:14.04


# 3. create

`imageTag` comes from step 1.

## parameters

| name | value | remark |
| ---- | ----- | ------ |
| `name` | `'myUbuntuTest'` | nothing special | 
| `ports`| `[HC(1234, 8888), HC(5678, 7777), HC(c=9999)]` | 0.0.0.0:5678->7777/tcp, <br> 0.0.0.0:1234->8888/tcp, <br> 0.0.0.0:32770->9999/tcp. <br> See `HC()` for more detail. |
| `volumes` | `[HCP('/tmp', '/mnt/host/tmp')]` | volume in host: `/tmp`, <br> volume in container: `/mnt/host/tmp`. <br> See `HCP()` for more detail. |

`HC()` is the `namedtuple` to hold host-to-container mapping pairs.
Attribute `h` refers to the host and `c` to the container. This is primarily used in assigning ports.
* `c` must be set, or we will just throw this tuple away before creating the container since it does not specify a valid port mapping.

`HCP()`, similar to `HC`, is the `namedtuple` to hold host-container-privilige tuple.
Attribute `h` for the host, `c` for the container, and `p` for the privilige. This is primarily used in assigning volumes and devices.
* In both cases, setting either `h` or `c` is must, or the tuple will simply be dropped. Setting one of `h` and `c` to `None` will map same path. 
* In the case of volumes, `p` can be assign to `rw` for read/write (default) or `ro` for read-only access.
* In the case of devices, `p` is usually set to `rwm` (default).

For both `HC` and `HCP`, we use some special tricks to allow default value. They are all default to `None`.
Refer to above specification for interpretation of `None` in each case.

*GPU binding is not tested here.*

In [49]:
container = docker.create(image=imageTag, 
           detach=True, 
           stdin_open=True, 
           tty=True, 
           command=None, 
           name='myUbuntuTest',
           user=None,
           ports=[HC(1234, 8888), HC(5678, 7777), HC(c=9999)],
           volumes=[HCP('/tmp', '/mnt/host/tmp')],
           #devices=[0,1,2,3]
           )
if container:
    print 'container is runing with id:'
    print container.get('Id')
else:
    print 'create container error'

container is runing with id:
e727d5e94599c307a12a7b531c48523b232d24db75fd774a94ee1f7704a6cb06


# 4. start
## *container_id* comes from step 3

In [50]:
container_id = container.get('Id')
response = docker.start(container=container_id)
if response:
    print 'container ', container_id, ' started'
else:
    print 'starting container error'

container  e727d5e94599c307a12a7b531c48523b232d24db75fd774a94ee1f7704a6cb06  started


# 5. check status

In [51]:
pses = docker.ps()
print 'Runing containers:'
print '##################'
for line in pses:
    print line['image'], line['name']

Runing containers:
##################
buildtest /myUbuntuTest
dsdgroup/dsd-console /dsd-console-devel-b7rfgpFN
dsdgroup/dsd-console /dsd-console-devel-bZ0cWaL5
wangjianyong/mxnet-cuda:7.5 /jpt-test


# 6. Stop 

In [52]:
container_id = container.get('Id')
response = docker.stop(container=container_id,
                       timeout=0)
if response:
    print 'container', container_id, 'stop succeed'
else:
    print '#######################'
    print 'container stop failed'

container e727d5e94599c307a12a7b531c48523b232d24db75fd774a94ee1f7704a6cb06 stop succeed


# 7. Remove container

In [53]:
container_id = container.get('Id')
response = docker.rm(container=container_id)
if response:
    print 'container ', container_id, 'removed succed'
else:
    print '#######################'
    print 'container removed failed'

container  e727d5e94599c307a12a7b531c48523b232d24db75fd774a94ee1f7704a6cb06 removed succed


# 8. Remove image

In [54]:
response = docker.rmi(image=imageTag,
                      force=False,
                      noprune=None)
if response:
    print 'image', imageTag, 'removed succeed'
else:
    print '#######################'
    print 'image removed failed'

image buildtest removed succeed
